## Cadenas de Markov ##

Una cadena de Markov es un sistema matemático que experimenta transiciones aleatorias en un conjunto de posibles estados, *sin memoria*: el siguiente estado depende solo del estado actual. El paseo aleatorio y las urnas de Polya o de Ehrenfest son ejemplos de cadenas de Markov.

En lo que sigue consideraremos cadenas de Markov discretas (un conjunto discreto de posibles estados) y homogéneas en el tiempo (las probabilidades de transición entre estados no dependen del tiempo). Cuando la cadena es finita (un conjunto finito de posibles estados) se puede describir su evolución por medio de su *matriz de transición* $P$, cuyas entradas son $p_{ij}=$ *probabilidad de transición del estado $i$ al estado $j$*. Nótese que la suma de las entradas de cualquier fila tiene que ser igual a 1. Obsérvese también que si $v^k$ es el vector fila que da las probabilidades de estar en cada estado en tiempo $k$ entonces $v^{k+1}=v^kP$, de forma que si trasponemos tenemos $(v^{k+1})^{\rm t}=P^{\rm t}(v^k)^{\rm t}$ y nos encontramos en una situación análoga a la considerada al estudiar modelos de dinámica de poblaciones. 

Queremos simular la evolución de sistemas descritos por cadenas de Markov. Para ello tendremos que ser capaces de, desde un estado $i$,  *sortear* a qué estado llegaremos en el paso siguiente, de acuerdo con las probabilidades dadas por la fila $i$ de la matriz $P$. 


**Ejercicio 1.-** De un cine, con tres salidas, $1/3$ de los espectadores sale por la salida $0$, $1/5$ por la $1$ y el resto, $7/15$, por la $2$. Se elige, al azar, un espectador, ¿por qué puerta saldrá? Esta situación se puede simular decidiendo conforme a la siguiente regla:
- se divide el intervalo $[0,1)$ en tres subintervalos de longitudes $1/3$, $1/5$ y $7/15$ respectivamente, es decir:
$$[0,1)= I_0\cup I_1\cup I_2=[0,\tfrac13)\cup [\tfrac 13,\tfrac13+\tfrac15) \cup [\tfrac13+\tfrac15,1)$$
- se decide la puerta de salida por el subintervalo en que se sitúe el número $t={\tt random}()$: si $t\in I_0$, la salida $0$; si $t\in I_1$, la $1$; por la $2$ en otro caso.

Es claro que de esta manera, se decidirá cada puerta de salida conforme a las probabilidades que nos han referido. La misma estrategia es válida para cualquier **vector** de probabilidades.

Codifica un programa *paso(v)* que dado un vector (fila) $v$ de probabilidad (es decir, de coordenadas no negativas y con suma igual a 1) genere de manera aleatoria un número entre $0$ y *len(v)-1*, teniendo cada uno de estos números una probabilidad *v[k]*.

*Sugerencia.-* Construye un vector $F$ con elementos $F[k]=\sum_{j=0}^k v[k]$. Genera un número aleatorio $a$ entre 0 y 1. La salida será el primer $k$ tal que $a<F[k]$.

In [175]:
def paso(v):
    '''v es un a lista de probabilidades-suman 1'''
    if sum(v)!=1:
        return None # error, no debe pasar
    F=[sum(v[:j]) for j in [1..len(v)]]
    num = random()
    j=0
    while num>F[j]:
        j+=1
    return j

In [176]:
paso([1/3,1/5, 7/15]) #, 1/3+1/5*1.

1

**Ejercicio 2.-** Supongamos que una cadena de Markov con estados $0,1,2$ tiene matriz $P$ de transición 

$$\begin{array}{|c|c|c|c|}& 0 & 1 & 2\\ \hline 0 & 0.1 & 0.2 & 0.7 \\ 1 & 0.9 & 0.1 & 0\\ 2 & 0.1 & 0.8 & 0.1\end{array}.$$

Sea $X_j=$ *el estado de la cadena en el tiempo $j$.* 


**a)** Simula la evolución del sistema durante $1000$ pasos suponiendo que incialmente está en el estado $1$. Contabiliza, por medio de un diccionario, el número de veces que el sistema ha estado en cada estado (*sugerencia.-* recuerda que el método * .row()* extrae una fila de una matriz).  Compara las proporciones del tiempo total que el sistema pasó en cada estado, que acabas de calcular,  con el autovector por la izquierda de la matriz $P$ correspondiente al autovalor 1, normalizado de forma que sea un vector de probabilidad. 

In [221]:
P = matrix(RDF, 3, [1/10,1/5,7/10,9/10,1/10,0,1/10,4/5,1/10])
show (P, P[1], P[2,1], P[1:], P[:,1])

[0.1 0.2 0.7]
[0.9 0.1 0.0]
[0.1 0.8 0.1] (0.9, 0.1, 0.0) 0.8 [0.9 0.1 0.0]
[0.1 0.8 0.1] [0.2]
[0.1]
[0.8]

In [238]:
M = 10^4
frec = dict([(j,0) for j in [0..2]])
estado = 1 #estado inicial
frec[1] = 1
for _ in xrange(M):
    estado = paso(P[estado])
    frec[estado]+=1
print(frec)

{0: 3719, 1: 3399, 2: 2883}


In [239]:
prop = vector([frec[j]/M for j in [0..2]])
prop.n()

(0.371900000000000, 0.339900000000000, 0.288300000000000)

In [240]:
v1 = vector((P.eigenvectors_left()[0][1])[0])
v1 = v1*1/sum(v1)
v1

(0.371559633027523, 0.33944954128440363, 0.2889908256880735)

**b)** Simula la evolución del sistema durante $1000$ pasos si se parte de un estado inicial elegido aleatoriamente según la distribución de probabilidades  siguiente: $P(X_0=0)=0.3$, $P(X_0=1)=0.4$ y $P(X_0=2)=0.3$. Contabiliza las veces que el sistema está en cada estado. ¿Ves alguna diferencia significativa con el caso anterior, en el que se partía de un estado determinado? *Sugerencia.-* Para el "sorteo" del estado inicial puedes usar también la funcion $paso$ del Ejercicio 1.

In [266]:
M = 10^3
frec = dict([(j,0) for j in [0..2]])
estado = paso([3/10, 4/10, 3/10]) #estado inicial
frec[estado] = 1
for _ in xrange(M):
    estado = paso(P[estado])
    frec[estado]+=1
print(frec)

{0: 369, 1: 348, 2: 284}


In [267]:
prop = vector(frec.values())/M
prop.n()

(0.369000000000000, 0.348000000000000, 0.284000000000000)

**c)** Estima experimentalmente la probabilidad de, partiendo del estado $0$ acabar en $5$ pasos en el estado $2$. Compara con el elemento $(0,2)$ de la matriz $P^5$.

In [273]:
M=10^3
fav = 0
for  _ in xrange(M):
    estado = 0
    for __ in xrange(5):
        estado = paso(P[estado])
    fav += (estado==2)
prob = fav / M*1.
print(prob)

0.242200000000000


In [276]:
print ((P^5)[0,2])
show(P^5)

0.238


[            0.33616             0.42584               0.238]
[ 0.3060000000000001  0.2914300000000001 0.40257000000000004]
[ 0.4940800000000001 0.28478000000000003             0.22114]

**d)** Estima experimentalmente la probabilidad de que el sistema empiece en el estado 0 y llegue al estado 2 en cinco pasos, suponiendo que el estado inicial se elige aleatoriamente como en el apartado **b)**. Compara el valor obtenido con el producto de $P(X_0=0)$ con el elemento $(0,2)$ de la matriz $P^5$.

In [281]:
M=10^4
fav = 0
for  _ in xrange(M):
    estado = paso([.3,.4,.3])
    if estado==0:
        for __ in xrange(5):
            estado = paso(P[estado])
        fav += (estado==2)
prob = fav / M*1.
print(prob, .3*(P^5)[0,2])

(0.0759000000000000, 0.0714000000000000)


**Ejercicio 3 (Un modelo simple de fecundidad).- **Se tienen 6 estados:</p>
<table border="0" cellspacing="5" cellpadding="12">
<tbody>
<tr>
<td>$E_0$: Ni&ntilde;as</td>
<td>$E_3$: Divorciadas</td>
</tr>
<tr>
<td>$E_1$: Solteras</td>
<td>$E_4$: Viudas</td>
</tr>
<tr>
<td>$E_2$: Casadas</td>
<td>$E_5$: $\Delta$</td>
</tr>
</tbody>
</table>

El último correspondería a la muerte o emigración, y se supone un *estado absorbente* (un estado del que no se puede salir). Se supone que la matriz de probabilidades de transición entre estados  es</p>
<p>$$\begin{array}{l|rrrrrr} & E_0&E_1&E_2&E_3&E_4&E_5\\ \hline E_0& 0.0 & 0.9 & 0.0 & 0.0 & 0.0 & 0.1 \\ E_1 &0.0 & 0.5 & 0.4 & 0.0 & 0.0 & 0.1 \\ E_2 &0.0 & 0.0 & 0.6 & 0.2 & 0.1 & 0.1 \\ E_3&0.0 & 0.0 & 0.4 & 0.5 & 0.0 & 0.1 \\ E_4&0.0 & 0.0 & 0.4 &0.0 &0.5 &0.1 \\ E_5 &0.0 &0.0 &0.0 &0.0 &0.0 & 1.0 \end{array}$$</p>

**a)** Estamos interesados en la duración media esperada en el estado $E_2$: Casadas, pues este corresponde al de máxima fecundidad.  Estima este valor experimentalmente simulando la línea de vida de $10^3$ mujeres desde el estado $E_0$ hasta el estado absorbente $E_5$, y calculando el tiempo medio transcurrido en el estado $E_2$.

In [282]:
P1 = matrix(RDF,6,[0,9/10,0,0,0,1/10,
                   0,1/2,4/10,0,0,1/10,
                  0,0,6/10,1/5,1/10,1/10,
                  0,0,4/10,1/2,0,1/10,
                  0,0,4/10,0,1/2,1/10,
                  0,0,0,0,0,1])
show(P1)

[0.0 0.9 0.0 0.0 0.0 0.1]
[0.0 0.5 0.4 0.0 0.0 0.1]
[0.0 0.0 0.6 0.2 0.1 0.1]
[0.0 0.0 0.4 0.5 0.0 0.1]
[0.0 0.0 0.4 0.0 0.5 0.1]
[0.0 0.0 0.0 0.0 0.0 1.0]

In [289]:
M = 10^3
fav = 0
for _ in xrange(M):
    estado = 0
    while estado != 5 :
        estado = paso(P1[estado])
        fav+=(estado==2)
print(u'Duración media en estado 2: '+ str(fav / M*1.))

Duración media en estado 2: 4.60400000000000


** b)** Otra cuestión de interés es el tiempo medio transcurrido hasta llegar al estado terminal (algo así como la *esperanza de vida*). Estima este valor experimentalmente simulando la vida de $10^3$ mujeres y calculando el tiempo medio transcurrido fuera del estado $E_5$.

In [290]:
M = 10^3
total = 0
for _ in xrange(M):
    estado = 0
    while estado != 5 :
        total+=(1)
        estado = paso(P1[estado])
print(u'Duración media antes del estado 5: '+ str(total / M*1.))

Duración media antes del estado 5: 9.74200000000000


**Ejercicio 4 (laberinto).-**  Se introduce una rata en el siguiente laberinto:
    
![Laberinto](./laberinto.png)

En la celda $7$ se encuentra la comida y en la $8$ recibe una descarga eléctrica. Al recibir la descarga o encontrar la comida, la rata deja de moverse (ambos se consideran estados absorbentes). Se supone, además, que la rata no aprende por dónde ha pasado de manera que se mueve por el recinto aleatoriamente.</p>

Se pide estimar experimentalmente: 
<ul>
<li>la probabilidad de encontrar la comida suponiendo que empieza en la celda $2$;</li>
<li>la probabilidad de sufrir una descarga suponiendo que se introduce al azar en el laberinto;</li>
<li>el tiempo medio empleado en llegar a la comida por una rata que empieza en $6$ y es capaz de llegar a la comida.</li>
</ul>

In [303]:
P2 = matrix(RDF,9)
P2[0,[1,2]] = P2[6,[4,5]] = 1/2
P2[1,[0,3,7]] = P2[4,[7,3,6]] = P2[2,[0,3,8]] = P2[5,[3,8,6]] = 1/3
P2[3,[1,2,4,5]] = 1/4
P2[8,8] = P2[7,7] = 1
show(P2.n(digits=2))

[0.00 0.50 0.50 0.00 0.00 0.00 0.00 0.00 0.00]
[0.33 0.00 0.00 0.33 0.00 0.00 0.00 0.33 0.00]
[0.33 0.00 0.00 0.33 0.00 0.00 0.00 0.00 0.33]
[0.00 0.25 0.25 0.00 0.25 0.25 0.00 0.00 0.00]
[0.00 0.00 0.00 0.33 0.00 0.00 0.33 0.33 0.00]
[0.00 0.00 0.00 0.33 0.00 0.00 0.33 0.00 0.33]
[0.00 0.00 0.00 0.00 0.50 0.50 0.00 0.00 0.00]
[0.00 0.00 0.00 0.00 0.00 0.00 0.00  1.0 0.00]
[0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00  1.0]

In [310]:
M = 10^3
total = 0
for _ in xrange(M) :
    estado = 2
    while estado!=8 and estado!=7:
        estado = paso(P2[estado])
    total += (estado==7)
print(u'Probabilidad de terminar en la comida saliendo'
      +' de la celda 2: \n\t' + str(total/M*1.) )

Probabilidad de terminar en la comida saliendo de la celda 2: 
	0.350000000000000


In [315]:
M = 10^3
total = 0
for _ in xrange(M) :
    estado = randint(0,8)# suponemos que se puede introducir 
                         # en la celda 8 también directamente
    while estado!=8 and estado!=7:
        estado = paso(P2[estado])
    total += (estado==8)
print(u'Probabilidad sufrir descarga saliendo de una'
      +' celda aleatoria: \n\t' + str(total/M*1.) )

Probabilidad sufrir descarga saliendo de una celda aleatoria: 
	0.505000000000000


In [339]:
M = 10^3
total = 0

for _ in xrange(M) :
    estado=0#inicialización a cualquier valor distinto de 7
    while estado!=7: #solo contamos exitosos, 
                     #si no llega, lo repetimos
        estado = 6
        count = 0
        while estado!=8 and estado!=7:
            count+=1
            estado = paso(P2.row(estado))
    total+=count
print(u'Pasos medios hechos por una rata exitosa que sale de '
      +'la celda 6: \n\t' + str(total/M*1.) )

Pasos medios hechos por una rata exitosa que sale de la celda 6: 
	5.94400000000000
